In [37]:
using ReadableRegex
using Plots
using Makie, CairoMakie
cd("./example")

In [41]:
module Runner
end

Main.Runner

In [57]:
function extractnotebook(notebook, notebookname=nothing)
    s = read(notebook, String)
    cells = split(s, "# ╔═╡ ");
    # The first cell and the final 3 are not used
    codes, contents, outputtag, celltype = [],[],[],[]
    for cell in cells[2:end-3]
        push!(codes, cell[1:36])
        push!(contents, cell[38:end])
        push!(outputtag, endswith(rstrip(cell),";") ? "hideoutput" : "showoutput")
        push!(celltype, cell[38:42] == "md\"\"\"" ? "markdown" : "code")
    end
    
    # Get order and view type
    r = either("# ╠","# ╟")
    sortedcells = split(cells[end],Regex(r))
    sortedcodes = [cell[4:39] for cell in sortedcells[2:end-2]]
    order = [findfirst(isequal(scode),codes) for scode in  sortedcodes[1:end]]
    view  = [occursin("═",c) ? "showcode" : "hidecode" for c in sortedcells[2:end-2]]
    # Matching running order
    view  = view[[findfirst(isequal(scode),sortedcodes) for scode in  codes]]

    # inferring the notebook name
    # base on the notebook file path.
    if notebookname === nothing
        notebookname = split(notebook,"/")[end]
        if endswith(notebookname,".jl")
            # removes the ".jl" in the end
            notebookname = notebookname[1:end-3]
        end
    end
    
    notebookdata = Dict(:codes => codes, :notebookname => notebookname,
                        :contents => contents, :outputtag=>outputtag,
                        :celltype => celltype,:order=> order,:view=>view)
    return notebookdata
end

function collectoutputs(notebookdata, notebookfolder="./")
    figureindex = Dict(:i => 0)
    runpath = pwd()
    cd(notebookfolder)
    outputs = []
    for (i, content) ∈ enumerate(notebookdata[:contents])
        if notebookdata[:celltype][i] == "code"
            if startswith(lstrip(content),"begin") && endswith(rstrip(content),"end")
                ex = :($(Meta.parse(strip(content))))
            else
                ex = :($(Meta.parse("begin\n"*content*"\nend")))
            end
            
            s = string(ex.args[end])
            if contains(s, Regex(either("PlutoUI.LocalResource","LocalResource")))
                if findfirst(Regex(look_for(one_or_more(ANY),after="(\"",before="\")")),s) === nothing
                    Runner.eval(ex)
                    pathvariable = s[findfirst(Regex(look_for(one_or_more(ANY),after="(",before=")")),s)]
                    imagepath = Runner.eval(Meta.parse(pathvariable))
                else
                    imagepath = s[findfirst(Regex(look_for(one_or_more(ANY),after="(\"",before="\")")),s)]
                end
                push!(outputs,(:image,imagepath))
            else
                io = IOBuffer();
                Base.invokelatest(show,
                    IOContext(io, :limit => true),"text/plain",
                    dispatch_output(Runner.eval(ex), notebookdata[:notebookname], runpath, figureindex));
                celloutput = String(take!(io))
                if celloutput == "nothing"
                    push!(outputs,nothing)
                elseif startswith(celloutput, "Plot{Plots.")
                    push!(outputs,
                    (:plot,notebookdata[:notebookname]*"_"*"figure"*string(figureindex[:i])*".png"))
                elseif startswith(celloutput, "FigureAxisPlot()")
                    push!(outputs,
                    (:plot,notebookdata[:notebookname]*"_"*"figure"*string(figureindex[:i])*".png"))
                else
                    push!(outputs,(:text, celloutput))
                end
            end
        else
            push!(outputs,nothing)
        end
    end
    cd(runpath)
    return outputs
end

function dispatch_output(command_eval::Makie.FigureAxisPlot, notebookname, runpath, figureindex)
    figureindex[:i]+=1
    save(runpath*"/build_latex/figures/"*notebookname*"_"*"figure"*string(figureindex[:i])*".pdf", command_eval)
    return command_eval
end

function dispatch_output(command_eval::Plots.Plot, notebookname, runpath, figureindex)
    figureindex[:i]+=1
    println(runpath)
    savefig(command_eval,runpath*"/build_latex/figures/"*notebookname*"_"*"figure"*string(figureindex[:i])*".png")
    return command_eval
end

function dispatch_output(command_eval, notebookname, runpath, figureindex)
   return command_eval 
end

function createstructure(path="./")
    folder = path*"/build_latex/"
    if !isdir(folder)
    mkpath(folder*"notebooks")
    mkpath(folder*"figures")
    else
        if !isdir(folder*"notebooks")
            mkpath(folder*"notebooks")
        end
        if !isdir(folder*"figures")
            mkpath(folder*"figures")
        end
    end
end


function notebooktolatex(notebook)
end

notebooktolatex (generic function with 1 method)

In [58]:
example = "./examplepluto.jl"
nb = extractnotebook(example)

Dict{Symbol, Any} with 7 entries:
  :codes        => Any["7b4b693d-c955-4c9b-97e8-56d28cf74e18", "bdad72d3-5744-4…
  :outputtag    => Any["showoutput", "showoutput", "showoutput", "showoutput", …
  :celltype     => Any["code", "code", "code", "markdown", "code", "code", "cod…
  :view         => ["showcode", "showcode", "hidecode", "showcode", "showcode",…
  :order        => [4, 1, 2, 5, 6, 7, 3, 8, 9, 10, 11, 12]
  :notebookname => "examplepluto"
  :contents     => Any["using PlutoUI\n\n", "\n\t begin\n\tusing Plots\n\ty(x) …

In [59]:
outputs = collectoutputs(nb,"./");

/home/davibarreira/MEGA/EMAp/PlutoLatexConverter.jl/example


┌ Warning: `LocalResource` **will not work** when you share the script/notebook with someone else, _unless they have those resources at exactly the same location on their file system_. 
│ 
│ ## Recommended alternatives (images)
│ 1. Go to [imgur.com](https://imgur.com) and drag&drop the image to the page. Right click on the image, and select "Copy image location". You can now use the image like so: `PlutoUI.Resource("https://i.imgur.com/SAzsMMA.jpg")`.
│ 2. If your notebook is part of a git repository, place the image in the repository and use a relative path: `PlutoUI.LocalResource("../images/cat.jpg")`.
└ @ PlutoUI /home/davibarreira/.julia/packages/PlutoUI/iYlHi/src/Resource.jl:80


In [60]:
outputs

12-element Vector{Any}:
 nothing
 (:plot, "examplepluto_figure1.png")
 nothing
 nothing
 (:text, "3-element Vector{Int64}:\n 10\n 10\n 10")
 (:text, "10-element Vector{Float64}:\n 0.8891274399292513\n 0.3028870622465778\n 0.5312253175365158\n 0.48801640701350935\n 0.3654495703054199\n 0.9094045080020947\n 0.2040115863189218\n 0.04123082110552434\n 0.8451105965259518\n 0.8527182703424292")
 (:text, "10-element Vector{Float64}:\n 1.8891274399292513\n 1.3028870622465778\n 1.5312253175365158\n 1.4880164070135093\n 1.36544957030542\n 1.9094045080020947\n 1.2040115863189218\n 1.0412308211055243\n 1.8451105965259518\n 1.8527182703424292")
 nothing
 (:plot, "examplepluto_figure2.png")
 (:image, "figure.svg")
 (:text, "\"plotexample.png\"")
 (:image, "plotexample.png")

In [111]:
nb[:contents][nb[:order]]

12-element Vector{Any}:
 "md\"\"\"\n# Chapter\n## Section\n### Subsection\n#### Subsubsection\nMy text here with *italics*, with **bold**, and a [link](https://davibarreira.github.io/).\n\nAdding some math expression here with \$x=10\$ and\n```math\nd(\\omega(t_0),\\omega(t_1)) \\leq \\int^{t_1}_{t_0}g(s) ds.\n```\n\nAdding some code like `plots`. Note that the `using plots`\n\"\"\"\n\n"
 "using PlutoUI\n\n"
 "\n\t begin\n\tusing Plots\n\ty(x) = sin(x)\n\tplot(y,\n\t\tcolor=:blue)\nend\n\n\n"
 "A = [10,10,10]\n\n"
 "x = rand(10);\n\n"
 "x .+ 1\n\n"
 "using Makie, CairoMakie\n\n"
 "set_theme!(theme_ggplot2())\n\n"
 "Makie.plot(x)\n\n"
 "PlutoUI.LocalResource(\"figure.svg\")\n\n"
 "figurepath = \"plotexample.png\";\n\n"
 "PlutoUI.LocalResource(figurepath)\n\n"

In [112]:
nb[:contents]

12-element Vector{Any}:
 "using PlutoUI\n\n"
 "\n\t begin\n\tusing Plots\n\ty(x) = sin(x)\n\tplot(y,\n\t\tcolor=:blue)\nend\n\n\n"
 "using Makie, CairoMakie\n\n"
 "md\"\"\"\n# Chapter\n## Section\n### Subsection\n#### Subsubsection\nMy text here with *italics*, with **bold**, and a [link](https://davibarreira.github.io/).\n\nAdding some math expression here with \$x=10\$ and\n```math\nd(\\omega(t_0),\\omega(t_1)) \\leq \\int^{t_1}_{t_0}g(s) ds.\n```\n\nAdding some code like `plots`. Note that the `using plots`\n\"\"\"\n\n"
 "A = [10,10,10]\n\n"
 "x = rand(10);\n\n"
 "x .+ 1\n\n"
 "set_theme!(theme_ggplot2())\n\n"
 "Makie.plot(x)\n\n"
 "PlutoUI.LocalResource(\"figure.svg\")\n\n"
 "figurepath = \"plotexample.png\";\n\n"
 "PlutoUI.LocalResource(figurepath)\n\n"

In [113]:
nb[:outputtag][1] == "showoutput"

true

In [115]:
notebook = "./build_latex/notebooks/"*nb[:notebookname]*".tex"
open(notebook, "w") do f
    write(f,"\\newpage\n")
    for i in nb[:order]
        if nb[:celltype][i] == "code" && nb[:view][i] == "showcode"
            write(f,"\n\\begin{lstlisting}[language=JuliaLocal, style=julia]\n")
            write(f, strip(nb[:contents][i]))
            write(f,"\n\\end{lstlisting}\n")
        end
        if nb[:celltype][i] == "code" && nb[:outputtag][i] == "showoutput"
            if outputs[i] === nothing
            elseif outputs[i][1] == :text
                write(f,"\n\\begin{verbatim}\n")
                write(f, outputs[i][2])
                write(f,"\n\\end{verbatim}\n")
            elseif outputs[i][1] == :plot
                write(f,"\n\\begin{figure}[H]\n")
                write(f,"\t\\centering\n")
                write(f,"\t\\includegraphics[width=0.8\\textwidth]{./figures/"*outputs[i][2]*"}\n")
                write(f,"\t\\label{fig:"*outputs[i][2]*"}\n")
                write(f,"\n\\end{figure}\n")
            end
        end
    end
#     write(f,"\\newpage\n")
end

In [92]:
function skiplines(io::IO, n)
    i = 1
    while i <= n
       eof(io) && error("File contains less than $n lines")
       i += read(io, Char) === '\n'
    end
end
    
# Function based on https://discourse.julialang.org/t/write-to-a-particular-line-in-a-file/50179
function writetext(file::String, text::String, linenumber::Integer, endline=true)
    f = open(file, "r+");
    if endline
        skiplines(f, linenumber);
        skip(f, -1)
    else
        skiplines(f, linenumber-1);
    end
    mark(f)
    buf = IOBuffer()
    write(buf, f)
    seekstart(buf)
    reset(f)
    print(f, text);
    write(f, buf)
    close(f)
end

function insertline(file::String, text::String, linenumber::Integer)
    if linenumber==1
        writetext(file, text*"\n", linenumber, false)
    else
        writetext(file, "\n"*text, linenumber-1)
    end
end

insertline (generic function with 1 method)

In [94]:
insertline(notebook,"inserted line", 2)

In [95]:
insertline(notebook,"inserted line", 1)

In [83]:
writetext(notebook, "testing",1, false)
writetext(notebook, "testing",1)

In [68]:
editline(notebook,"DAIVII",2)

In [114]:
read("../templates/frontmatter/copyright.tex", String)

"% Copyright page\n\n{\\small\n\\setlength{\\parindent}{0em}\\setlength{\\parskip}{1em}\n\n~\n\n\\vfill\n\nCopyright \\copyright{} 2021 \\authorname\n\nAll rights reserved. No part of this publication may be reproduced, stored or transmitted in any form or by any means, electronic, mechanical, photocopying, recording, scanning, or otherwise without written permission from the publisher. It is illegal to copy this book, post it to a website, or distribute it by any other means without permission.\n\nFirst edition, \\editionyear{}\n\nISBN \\isbn{}  % see main.tex\n\nPublished by \\publisher{}\n}\n"

In [34]:
function altere(v)
    x = Dict(:figureindex => 1)
    println(x)
    f!(x)
    println(x)
    
end

function f!(x)
    x[:figureindex] += 1
end

f! (generic function with 5 methods)

In [35]:
altere(1)

Dict(:figureindex => 1)
Dict(:figureindex => 2)


In [28]:
parse(Int,x)

2